### 自動類別 

有些模型與 Tokenizer 第一時間可能無法判別的使用什麼架構，或是想寫出相容性更廣泛的方案碼，那就可以使用 Transformers 的 Auto Class 來進行讀取。

* 比較差異

| 項目       | `LlamaTokenizer` | `LlamaTokenizerFast`                   |
| -------- | ---------------- | -------------------------------------- |
| 📦 實作方式  | Python 純實作       | Rust 背後實作（透過 `tokenizers` 套件）          |
| ⚡ 效能     | 慢（尤其處理大量資料時）     | 快很多（數倍於慢速版本）                           |
| ✅ 功能完整性  | 全功能，易除錯          | 幾乎完全相容，速度快                             |
| 🧩 所需依賴  | `transformers`   | `transformers` + `tokenizers`（Rust 編譯） |
| 📉 記憶體效率 | 通常略高             | 更優，效能更佳                                |
| ❗ 適用情境   | 除錯、輕量任務、小型批次     | 推論部署、訓練大量資料                            |
* 高速推論或訓練 --> `LlamaTokenizerFast`
* 除錯特殊 tokenizer 行為 --> `LlamaTokenizer`（慢速）
* 只想「它能跑」 --> `AutoTokenizer`（自動挑快的） 



1. 判別的使用架構：

In [1]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelCls
from transformers import AutoTokenizer as TkCls

model_path = "TheBloke/Llama-2-7b-chat-fp16"
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkCls.from_pretrained(model_path)

print("\n--------------------------------------------------------\n")
print(type(model))  # <class 'transformers...LlamaForCausalLM'>
print(type(tk))     # <class 'transformers...LlamaTokenizerFast'>

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. 


--------------------------------------------------------

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>


如果不想用快速版本的 Tokenizer 可以把 use_fast 參數設定成 False：

In [2]:
tk: TkCls = TkCls.from_pretrained(model_path, use_fast=False)
print(type(tk))     # <class 'transformers...LlamaTokenizer'>

<class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>


2. 大部分的方法都來自 `PreTrainedModel` 與 `PreTrainedTokenizer` 這兩個型別，因此可以給出型別註記：(別名)
    * 自動完成回來了。
    * 最新版的 Transformers 套件中，`AutoTokenizer.from_pretrained()` 已經會自動把回傳物件標記為 `PreTrainedTokenizer` 型別。

In [3]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelImp
from transformers import AutoTokenizer as TkImp
from transformers import PreTrainedModel as ModelCls
from transformers import PreTrainedTokenizer as TkCls

model_path = "TheBloke/Llama-2-7b-chat-fp16"
model: ModelCls = ModelImp.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkImp.from_pretrained(model_path)

print("\n--------------------------------------------------------\n")
print(type(model))  # <class 'transformers...LlamaForCausalLM'>
print(type(tk))     # <class 'transformers...LlamaTokenizerFast'>

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parame


--------------------------------------------------------

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>
